In [127]:
# First make YAML from dataset

# Then maybe use jinja or something to render as HTML



In [128]:
import pandas as pd
from datetime import datetime as dt
import re

YEAR_RANGE = (1900, 1950)

In [129]:
def remove(row):
    try:
        date = dt.strptime(row.Date, '%Y-%m-%d')
    except:
        return True
    
    if YEAR_RANGE and date.year >= YEAR_RANGE[0] and date.year <= YEAR_RANGE[1]:
        pass # in the range!
    else:
        return True

    if row['Exclude from visualization'] or row['Unsure whether drag artist']:
        return True
    
    no_city = row['City'] == ''
    no_performer = row['Performer'] == ''
    no_venue = row['Venue'] == ''
    unnamed_performer = 'unnamed' in row['Performer'].lower()
    
    if no_city and no_performer and no_venue:
        return True
    
    if unnamed_performer:
        return True

    return False

def extract_addresses_dict(normalized_df):
    addresses = {}
    rows_with_addresses = normalized_df[normalized_df['Address']!='']
    warnings = []
    for x in zip(rows_with_addresses['Date'], rows_with_addresses['Source'], rows_with_addresses['Venue'], rows_with_addresses['Address']):
        date, source, venue, address = x
        if venue == '':
            warnings.append(address)
        else:
            if not venue in addresses:
                addresses[venue] = {}
            if not source in addresses[venue]:
                addresses[venue][source] = address
    if len(warnings):
        print(f'Warning: {len(warnings)} Venues with no names have addresses:')
        print('- ' + '- '.join(warnings))

    return addresses

def get_comments(df, comment_field='Comment on edge: revue', match_field='Revue', transform=None):
    comments = {}
    rows_with_comments = df[df[comment_field]!='']
    warnings = []
    for x in zip(rows_with_comments['Date'], rows_with_comments['Source'], rows_with_comments[match_field], rows_with_comments[comment_field]):
        date, source, match, comment = x
        comment = str(comment).strip()
        if transform:
            comment = transform(comment)
        if match == '':
            warnings.append(str(comment)[:40]+'...')
        else:
            if not match in comments:
                comments[match] = {}
            if not source in comments[match]:
                comments[match][source] = comment
    if len(warnings):
        print(f'Warning: {len(warnings)} mentions in `{comment_field}` with no value have comments:')
        print('- ' + '\n- '.join(warnings))

    return comments

def get_revue_comments_dict(df):
    return get_comments(df, 'Comment on edge: revue', 'Revue')

def get_performer_comments_dict(df):
    return get_comments(df, 'Comment on node: performer', 'Performer')

def get_venue_comments_dict(df):
    return get_comments(df, 'Comment on node: venue', 'Venue')

def get_city_comments_dict(df):
    return get_comments(df, 'Comment on node: city', 'City')

def get_true_value(row, type):
    if type == 'source':
        if row['Source clean'] != '':
            return row['Source clean']
        return row['Source']
    if type == 'performer':
        if row['Normalized performer'] != '':
            return row['Normalized performer']
        if row['Performer first-name'] != '' and row['Performer last-name'] != '':
            return row['Normalized performer']
        return row['Performer']
    if type == 'city':
        if row['Normalized City'] != '':
            return row['Normalized City']
        return row['City']
    if type == 'revue':
        if row['Normalized Revue Name'] != '':
            return row['Normalized Revue Name']
        return row['Revue name']
    if type == 'venue':
        if row['Normalized Venue'] != '':
            return row['Normalized Venue']
        return row['Venue']
    raise NotImplementedError(f'type `{type}` is not yet implemented')

def find_ref(row, eima=True):
    source = row['Source']
    source += ' ' + row['EIMA']
    source += ' ' + row['Search (newspapers.com)']
    source += ' ' + row['Source clean']
    
    is_eima = 'eima' in source.lower() or 'variety' in source.lower() or 'billboard' in source.lower()
    has_ref = re.search(r'(\d{7,10})', source)
    refs = list(set(re.findall(r'(\d{7,10})', source)))
    if has_ref and eima and is_eima:
        return '|'.join(refs)
    
    if has_ref and not eima and not is_eima:
        return '|'.join(refs)

    return ''
    

df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT0E0Y7txIa2pfBuusA1cd8X5OVhQ_D0qZC8D40KhTU3xB7McsPR2kuB7GH6ncmNT3nfjEYGbscOPp0/pub?gid=0&single=true&output=csv')
df = df.fillna('')
df = df.replace('—', '')
df = df.replace('—*', '')

# First, set up our references to EIMA and newspapers.com
df['EIMA'] = df.apply(lambda row: find_ref(row), axis=1)
df['Newspapers.com'] = df.apply(lambda row: find_ref(row, False), axis=1)

# Normalize dataframe
df['Source'] = df.apply(lambda row: get_true_value(row, 'source'), axis=1)
df['Venue'] = df.apply(lambda row: get_true_value(row, 'venue'), axis=1)
df['Performer'] = df.apply(lambda row: get_true_value(row, 'performer'), axis=1)
df['City'] = df.apply(lambda row: get_true_value(row, 'city'), axis=1)
df['Revue'] = df.apply(lambda row: get_true_value(row, 'revue'), axis=1)

# Extract "node" information
addresses = extract_addresses_dict(df)
revue_comments = get_revue_comments_dict(df)
performer_comments = get_performer_comments_dict(df)
venue_comments = get_venue_comments_dict(df)
city_comments = get_city_comments_dict(df)
edge_comments = get_comments(df, 'Edge Comment', 'Source')
legal_names = get_comments(df, 'Legal name', 'Performer')
ages = get_comments(df, 'Alleged age', 'Performer', lambda x: int(float(x)))
birth_years = get_comments(df, 'Assumed birth year', 'Performer', lambda x: int(float(x)))
eima_links = get_comments(df, 'EIMA', 'Source')
newspaper_links = get_comments(df, 'Newspapers.com', 'Source')

# Edge booleans
blackface_performers = get_comments(df, 'Blackface', 'Performer', lambda x: bool(x))
sepia_performers = get_comments(df, 'Sepia', 'Performer', lambda x: bool(x))
fan_dance_performers = get_comments(df, 'Fan dancer/Sally Rand', 'Performer', lambda x: bool(x))
exotic_dancers = get_comments(df, 'Exotic/erotic/oriental dancer/Gypsy', 'Performer', lambda x: bool(x))
has_image = get_comments(df, 'Has image', 'Performer', lambda x: bool(x))

# Drop filtered data
df['remove'] = df.apply(lambda row: remove(row), axis=1)
df = df.drop(df[df['remove']==True].index)

# Drop columns
df = df.drop(['Category', 'EIMA', 'Newspapers.com', 'Search (newspapers.com)', 'Blackface', 'Sepia', 'Fan dancer/Sally Rand', 'Exotic/erotic/oriental dancer/Gypsy', 'Has image', 'Legal name', 'Alleged age', 'Assumed birth year', 'Search (fulton)', 'Imported from former archive', 'Edge Comment', 'Comment on node: performer', 'Comment on node: venue', 'Comment on node: city', 'Comment on edge: revue', 'Exclude from visualization', 'Address', 'Unsure whether drag artist', 'Source clean', 'Normalized performer', 'Performer first-name', 'Performer last-name', 'Normalized Venue', 'Normalized City', 'Normalized Revue Name', 'Revue name', 'remove'], axis=1)

- 925 East Main Street
- "A treat will be offered to the patrons ...
- Touring circus, LaTour was with them las...
- "About September 15, Barnett Hyman will ...
- "Sensational Floor Show"...
- "Sensational Floor Show"...
- featuring "Several Female Impersonators"...
- featuring "Several Female Impersonators"...
- featuring "Several Female Impersonators"...
- "Here a corking good floor show is prese...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- Paris-Harlem Nite club and ball room, "s...
- "Opening Wednesday Nite, July 3rd", "Pre.

In [130]:
# Time to render some files

from jinja2 import Environment, FileSystemLoader
from slugify import slugify
from pathlib import Path
import os

In [131]:
def keyshift(dictionary, key, diff):
    if key in dictionary:
        token = object()
        keys = [token]*(diff*-1) + sorted(dictionary) + [token]*diff
        newkey = keys[keys.index(key)+diff]
        if newkey is token:
            return None
        else:
            return newkey
    else:
        return None


def slugify_column(df, column='Performer'):
    if not column == 'Performer':
        all_values = list(sorted(set([x for x in df[column] if x and not x.startswith('—')])))
    else:
        all_values = list(sorted(set([x for x in df[column] if x]))) # we have to include the ones that start with — here
    values_dict = {}
    for value in all_values:
        done = False
        i = 0
        while not done:
            if i == 0:
                if not slugify(value) in values_dict:
                    values_dict[slugify(value)] = value
                    done = True
                else:
                    i += 1
            else:
                print('Warning: Multiple values with the same value. This should not happen:', value)
                if not f'{slugify(value)}-{i}' in values_dict:
                    values_dict[f'{slugify(value)}-{i}'] = value
                    done = True
                else:
                    i += 1
    return {v: k for k, v in values_dict.items()} # reversed
    

In [132]:
def get_venue_slug(venue):
    return ALL_VENUES[venue]

def get_performer_slug(performer):
    return ALL_PERFORMERS[performer]


TEMPLATE_DIR = '/Users/kallewesterling/Repositories/kallewesterling/dissertation/drag-data-browser/templates/'
OUTPUT_DIR = '/Users/kallewesterling/Repositories/kallewesterling/dissertation/drag-data-browser/docs/'
BASE_URL = '/drag-data-browser/'

e = Environment(loader=FileSystemLoader(TEMPLATE_DIR))

ALL_YEARS = list(range(YEAR_RANGE[0], YEAR_RANGE[1]))
ALL_PERFORMERS = slugify_column(df, 'Performer')
ALL_VENUES = slugify_column(df, 'Unique venue')
ALL_CITIES = slugify_column(df, 'City')

e.globals['get_venue_slug'] = get_venue_slug
e.globals['get_performer_slug'] = get_performer_slug
e.globals['ALL_YEARS'] = ALL_YEARS
e.globals['ALL_PERFORMERS'] = ALL_PERFORMERS
e.globals['ALL_VENUES'] = ALL_VENUES
e.globals['ALL_CITIES'] = ALL_CITIES
e.globals['BASE_URL'] = BASE_URL

In [133]:
artist_template = e.get_template('artist.html')


performers_active_dates_overview = {}

for performer, row in df.groupby(['Performer']):
    if not performer:
        continue

    html_file = os.path.join(OUTPUT_DIR, 'performer', ALL_PERFORMERS[performer], 'index.html')
    if not Path(html_file).parent.exists():
        Path(html_file).parent.mkdir(parents=True)
    
    full_venues = list(set([x for x in row['Unique venue'] if x and not x.startswith('—')]))
    full_venues = {x: ALL_VENUES[x] for x in full_venues}

    cities = list(set([x for x in row['City'] if x]))
    years_active = list(set(([x.year for x in pd.to_datetime(row['Date'])])))
    appears_at = list(set([x for x in row['Venue'] if x]))
    
    performers_active_dates_overview[performer] = years_active

    next_performer = keyshift(ALL_PERFORMERS, performer, +1)
    prev_performer = keyshift(ALL_PERFORMERS, performer, -1)
    if next_performer:
        next_performer = {'label': next_performer, 'url': BASE_URL + 'performer/' + ALL_PERFORMERS[next_performer]}
    if prev_performer:
        prev_performer = {'label': prev_performer, 'url': BASE_URL + 'performer/' + ALL_PERFORMERS[prev_performer]}
    
    text = artist_template.render(data={
        'name': performer,
        'years_active': years_active,
        'full_venues': full_venues,
        'cities': cities,
        'in_blackface': blackface_performers.get(performer, {}),
        'sepia_performer': sepia_performers.get(performer, {}),
        'fan_dancer': fan_dance_performers.get(performer, {}),
        'exotic_dancer': exotic_dancers.get(performer, {}),
        'images': has_image.get(performer, {}),
        'comments': performer_comments.get(performer, {}),
        'legal_name': legal_names.get(performer, {}),
        'age': ages.get(performer, {}),
        'birth_year': birth_years.get(performer, {}),
        'relative': {
            'next': next_performer,
            'prev': prev_performer
        }
    })

    with open(html_file, 'w+') as f:
        f.write(text)
        
        
        
########################


venue_template = e.get_template('venue.html')


venues_active_dates_overview = {}

for venue, row in df.groupby(['Unique venue']):
    if not venue or venue.startswith('—'):
        continue

    html_file = os.path.join(OUTPUT_DIR, 'venue', ALL_VENUES[venue], 'index.html')
    if not Path(html_file).parent.exists():
        Path(html_file).parent.mkdir(parents=True)

    associated_performers = list(set([x for x in row['Performer'] if x]))
    associated_performers = {x: ALL_PERFORMERS[x] for x in associated_performers}
    years_active = list(set(([x.year for x in pd.to_datetime(row['Date'])])))
    
    venues_active_dates_overview[venue] = years_active
    
    next_venue = keyshift(ALL_VENUES, venue, +1)
    prev_venue = keyshift(ALL_VENUES, venue, -1)
    if next_venue:
        next_venue = {'label': next_venue, 'url': BASE_URL + 'venue/' + ALL_VENUES[next_venue]}
    if prev_venue:
        prev_venue = {'label': prev_venue, 'url': BASE_URL + 'venue/' + ALL_VENUES[prev_venue]}
    
    text = venue_template.render(data={
        'name': venue,
        'years_active': years_active,
        'associated_performers': associated_performers,
        'addresses': addresses.get(venue, {}),
        'comments': venue_comments.get(venue, {}),
        'relative': {
            'next': next_venue,
            'prev': prev_venue,
        }
    })
    
    with open(html_file, 'w+') as f:
        f.write(text)
        


home_template = e.get_template('home.html')

html_file = os.path.join(OUTPUT_DIR, 'index.html')
if not Path(html_file).parent.exists():
    Path(html_file).parent.mkdir(parents=True)

text = home_template.render(data={
    'xxx': 'xxx'
})

with open(html_file, 'w+') as f:
    f.write(text)







import itertools
MIN = min(list(itertools.chain.from_iterable(venues_active_dates_overview.values())))
MAX = max(list(itertools.chain.from_iterable(venues_active_dates_overview.values())))
        
venue_list_template = e.get_template('venue-list.html')

html_file = os.path.join(OUTPUT_DIR, 'venue', 'index.html')
if not Path(html_file).parent.exists():
    Path(html_file).parent.mkdir(parents=True)

text = venue_list_template.render(data={
    'venues_active_dates_overview': venues_active_dates_overview,
    'venues_years_range': [x for x in range(MIN,MAX)]
})

with open(html_file, 'w+') as f:
    f.write(text)







import itertools
MIN = min(list(itertools.chain.from_iterable(performers_active_dates_overview.values())))
MAX = max(list(itertools.chain.from_iterable(performers_active_dates_overview.values())))
        
performer_list_template = e.get_template('performer-list.html')

html_file = os.path.join(OUTPUT_DIR, 'performer', 'index.html')
if not Path(html_file).parent.exists():
    Path(html_file).parent.mkdir(parents=True)

text = performer_list_template.render(data={
    'performers_active_dates_overview': performers_active_dates_overview,
    'performer_years_range': [x for x in range(MIN,MAX)]
})

with open(html_file, 'w+') as f:
    f.write(text)


In [73]:
# SEPARATE THING: FIXING DATES

In [74]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT0E0Y7txIa2pfBuusA1cd8X5OVhQ_D0qZC8D40KhTU3xB7McsPR2kuB7GH6ncmNT3nfjEYGbscOPp0/pub?gid=0&single=true&output=csv')

In [122]:
for ix, row in df.iterrows():
    if re.search(r'(January|February|March|April|May|June|July|August|September|October|November|December)', row['Source clean']):
        print(row['Source clean'])
        continue
    
    date = str(row.Date).replace('?', '')
    try:
        date = pd.to_datetime(date)
        if not date or pd.isnull(pd.to_datetime(date)):
            # print('error interpreting date', row.Date)
            print(row['Source clean'])
            continue
    except:
        #print('error interpreting date', row.Date)
        print(row['Source clean'])
        continue
    
    new_date = date.strftime(f'%B {date.day}, %Y')
    
    page_number = None
    g = re.search(r', (\d{1,3})$', row['Source clean'])
    if g:
        page_number = g.groups()[0]
    if page_number:
        print(row['Source clean'].replace(f', {page_number}', '') + ', ' + new_date + ', ' + page_number)
    else:
        print(row['Source clean'] + ', ' + new_date)

- obit, August 11, 1933
- fm27c
- fm27c
- fm27c
- fm27c
- fm27c
- fm27c
- fm28d, November 1, 1948
- fm28d, November 1, 1948
- fm28d, November 1, 1948
- fm28d, September 29, 1950
- fm28d, September 29, 1950
- fm28d, September 29, 1950
- fm28d, September 29, 1950
- obituary, May 1, 1933
- obituary, August 1, 1933
- hf-0833, August 12, 1933
—————————, June 14, 1924
—————————, May 18, 1935
—————————, December 28, 1935
—————————, December 28, 1935
—————————, December 28, 1935
—————————, December 28, 1935
—————————, April 29, 1936
—————————, April 29, 1936
—————————, April 29, 1936
—————————, April 29, 1936
—————————, October 9, 1936
—————————, November 27, 1936
—————————, January 16, 1937
—————————, September 4, 1937
—————————, September 4, 1937
—————————, April 13, 1938
—————————, April 20, 1938
—————————, April 27, 1938
—————————, May 4, 1938
—————————, May 7, 1938
—————————, May 7, 1938
—————————, May 7, 1938
—————————, May 7, 1938
—————————, May 7, 1938
—————————, January 12, 1940
—————

The Kansas City Times, September 21, 1935, 26
The Kansas City Times, September 21, 1935, 26
The Kansas City Times, September 21, 1935, 26
The Kansas City Times, April 11, 1936, 26
The Kansas City Times, April 11, 1936, 26
The Kansas City Times, April 11, 1936, 26
The Kansas City Times, April 11, 1936, 26
The Kansas City Times, April 11, 1936, 26
The Muscatine Journal, March 27, 1936, 11
The Pittsburgh Courier, April 17, 1937, 9
The Salt Lake Tribune, September 19, 1936, 22
The Salt Lake Tribune, September 19, 1936, 22
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend Tribune, September 12, 1936, 5
The South Bend T

The Herald-Press, July 20, 1936, 2
The Herald-Press, July 23, 1937, 2
The Herald-Press, July 23, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 26, 1937, 2
The Herald-Press, July 30, 1937, 2
The Journal Times, May 1, 1952, 28
The Journal Times, May 9, 1952, 25
The Kansas City Times, 8	649739903, November 30, 1935
The Kansas City Times, 8	649739903, November 30, 1935
The Kansas City Times, 8	649739903, November 30, 1935
The Kansas City Times, 8	649739903, November 30, 1935
The Kansas City Times, 8	649739903, November 30, 1935
The Miami Herald, April 8, 1982, 2
The Miami Herald, June 6, 1947

Syracuse Journal, April 12, 1935, 45
Syracuse Journal, April 12, 1935, 45
Syracuse Journal, April 12, 1935, 45
Syracuse Journal, April 12, 1935, 45
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 19, 1935, 37
Syracuse Journal, April 20, 1935, 16
Syracuse Journal, April 20, 1935, 16
Syracuse Journal, April 20, 1935, 16
Syracuse Journal, April 23, 1935, 22
Syracuse Journal, April 23, 1935, 22
Syracuse Journal, April 23, 1935, 22
Syracuse Journal, April 23, 1935, 22
Syracuse Journal, April 26, 1935, 37
Syracuse Journal, August 1, 1935, 30
Syracuse Journal, August 8, 1935, 37
Syracuse Journal, August 8, 1935, 37
Syracuse Journal, March 10, 1935, 23
Syracuse Journal, March 10, 1935, 23
Syracuse Journal, March 10, 1935, 23
S

Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 15, 1936, 9
Lansing State Journal, July 18, 1936, 7
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Lansing State Journal, July 25, 1936, 6
Muncie Evening Press, March 29, 1930, 5
New York Daily News, April 18, 1940, 47
New York Daily News, April 20, 1933, 17
New York Daily News, April 20, 1947, 91
New York Daily News, April 22, 1939, 22


Long Island Daily Press, July 14, 1934, 5
Long Island Daily Press, July 14, 1934, 5
Long Island Daily Press, July 14, 1934, 5
Long Island Daily Press, July 14, 1934, 5
Long Island Daily Press, July 14, 1934, 5
Long Island Daily Press, July 14, 1934, 5
Long Island Daily Press, July 21, 1934, 5
Long Island Daily Press, July 21, 1934, 5
Long Island Daily Press, July 21, 1934, 5
Long Island Daily Press, July 21, 1934, 5
New Britain Herald, February 11, 1927, 29
New Britain Herald, February 11, 1927, 29
New York Daily News, August 11, 1944, 19C
New York Daily News, August 11, 1944, 27B
New York Daily News, December 1, 1934, 25
New York Daily News, December 2, 1933, 23
New York Daily News, December 4, 1947, 80
New York Daily News, December 9, 1933, 27
New York Daily News, February 27, 1942, 4
New York Daily News, February 4, 1943, 47
New York Daily News, February 7, 1934, 37
New York Daily News, February 9, 1921, 11
New York Daily News, January 1, 1939, 28C
New York Daily News, January 1, 19

Jamestown Evening Journal, August 9, 1935, 14
Jamestown Evening Journal, August 9, 1935, 14
Jamestown Evening Journal, August 9, 1935, 14
Kansas City Times, July 6, 1967, page unknown
Kansas City Times, July 6, 1967, page unknown
New York Daily News, February 20, 1947, 3; 30
New York Herald Tribune, January 30, 1931, 36
Pensacola News Journal, February 24, 1950, 12
Pensacola News Journal, February 24, 1950, 12
Pensacola News Journal, February 24, 1950, 12
Pensacola News Journal, February 24, 1950, 12
Pensacola News Journal, February 24, 1950, 12
Pittsburgh Post-Gazette, December 3, 1941, 12
Pittsburgh Post-Gazette, December 3, 1946, 18
Pittsburgh Post-Gazette, January 22, 1946, 13
Pittsburgh Post-Gazette, October 26, 1933, 10
Portland Press Herald, November 13, 1949, 10B
Portland Press Herald, November 13, 1949, 10B
Portland Press Herald, November 13, 1949, 10B
Portland Press Herald, November 13, 1949, 10B
Portland Press Herald, November 13, 1949, 10B
Portland Press Herald, November 13

The Newark, Ohio Newark Advocate, September 17, 1954, 10
The Newark, Ohio Newark Advocate, September 23, 1954, 14
The News Journal (Wilmington, Delaware)July 31, 1934, 24
The Ogden, Utah Ogden Standard-Examiner, May 31, 1943, 9
The Pinedale Roundup [Pinedale, Wyoming], August 3, 1940
The Pinedale Roundup [Pinedale, Wyoming], August 3, 1940
The Pinedale Roundup [Pinedale, Wyoming], August 3, 1940
The Pinedale Roundup [Pinedale, Wyoming], August 3, 1940
The Scranton, Pennsylvania Tribune, January 24, 1940, 36
The Sikeston, Missouri Daily Standard, August 2, 1957, 3
The Vineland, New Jersey Daily Journal, July 11, 1928, 8
The Vineland, New Jersey Daily Journal, March 8, 1941, 3
Variety, March 9, 1960, 67 (Reviews: Apollo. N. Y Jose.)
Variety, March 9, 1960, 67 (Reviews: Apollo. N. Y Jose.)
Variety, March 9, 1960, 67 (Reviews: Apollo. N. Y Jose.)
Variety, March 9, 1960, 67 (Reviews: Apollo. N. Y Jose.)
Variety, March 9, 1960, 67 (Reviews: Apollo. N. Y Jose.)
Variety, March 9, 1960, 67 (Rev